In [56]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


293

In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/conference-experment', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-05-01 00:44:34.134201: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 00:44:34.167087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
reload(config)
reload(processed)
tokenized_data,loaded_data,raw_data = processed.tokenize_and_load_datasets()


Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

Map:   0%|          | 0/69615 [00:00<?, ? examples/s]

Map:   0%|          | 0/7735 [00:00<?, ? examples/s]

In [3]:
dataset= raw_data['source']
dataset

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 8335
    })
})

In [4]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
init(model)

In [6]:
from transformers import AdamW
from adapters import AdapterConfig, SeqBnInvConfig
from adapters.composition import Stack
adapter_name= "unipelt_source_mlm"

adapter_config = AdapterConfig.load("unipelt")

model.add_adapter(adapter_name, config = adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter(adapter_name)
model.active_adapters = adapter_name

fn.print_trainable_parameters(model)

trainable params: 6334520 || all params: 72727922 || trainable%: 8.709887242481642


In [7]:
model.adapter_summary(as_dict=True)

[{'name': 'unipelt_source_mlm',
  'architecture': 'union',
  'active': True,
  '#param': 5742392,
  'train': True,
  '%param': 8.653018072753925},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [8]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(dataset,tokenizer)
tokenized_dataset

Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 8335
    })
})

In [9]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset['train'][444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [10]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [11]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 24810
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2745
    })
})

In [12]:
print(tokenizer.decode(lm_datasets['test'][1]["input_ids"]))
print(tokenizer.decode(lm_datasets['test'][1]['labels']))
print(len(lm_datasets['test'][-1]['labels']))


, but it was with very good reason. [SEP] [CLS] okay well i'm in east plano i'm out in las rios so it's bound to be different over here but yeah yeah [SEP] i enjoy being in las rios. [SEP] [CLS] and killed them well it was two stores side by side [SEP] it brought them to life, the fact those stores were separated. [SEP] [CLS] pickup so they did say that you had to put it out within i think it was uh three weeks after christmas and [SEP] to be picked up, it has to be placed outside 3 weeks after xmas. [SEP] [CLS] i chased a bunch of kids down on
, but it was with very good reason. [SEP] [CLS] okay well i'm in east plano i'm out in las rios so it's bound to be different over here but yeah yeah [SEP] i enjoy being in las rios. [SEP] [CLS] and killed them well it was two stores side by side [SEP] it brought them to life, the fact those stores were separated. [SEP] [CLS] pickup so they did say that you had to put it out within i think it was uh three weeks after christmas and [SEP] to be pi

In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets['train'][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] yeah [MASK] they just they never stop [MASK] things to [MASK] [SEP] they keep changing it. [SEP] [CLS] and but uh in most places yeah they got forty five kids at best i [MASK] [SEP] [MASK] got five kids at most in remove places. [SEP] [CLS] [MASK] i can get [MASK] filter off just by laying down underneath the car i can't crawl under it the clearance isn't enough but enough to [MASK] under and get the filter off [MASK]'s not a problem [SEP] there isn't enough clearance under the car for me. [SEP] [CLS] bond this house that i have [MASK] just [MASK] three bedroom ranch and i can tell you there's nothing'

'>>> around it [MASK] foundation all the way round it not a shrub [MASK] a bush [SEP] i have a three bedroom ranch with no garden. all you can see is just [MASK] cement foundation all the way around. [SEP] [CLS] there's a big difference there [SEP] [MASK] is a significant [MASK] there but it works in [MASK] [MASK]. [SEP] [CLS] but [MASK] guess it automatic [MASK] guess it's 

In [14]:
reload(fn)
samples = [lm_datasets['train'][i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] yeah they they just they never stop doing things to it [SEP] they [MASK] changing it [MASK] [SEP] [CLS] and but uh in most places yeah they [MASK] forty five kids at best i guess [SEP] they got five kids at [MASK] [MASK] those places. [SEP] [CLS] well [MASK] [MASK] get the filter off just by laying down underneath the car i can't crawl under it the clearance isn [MASK] [MASK] enough but enough to reach under and get the filter off [MASK] [MASK] s not a problem [SEP] there isn't enough clearance under the car for me. [SEP] [CLS] yeah this house that i have is just a three bedroom [MASK] and i can tell [MASK] there's [MASK]'

'>>> [MASK] it cement foundation all the way round it not a shrub not a bush [SEP] i have a three bedroom ranch [MASK] no garden. all you [MASK] see is just the cement foundation all the [MASK] around. [SEP] [CLS] there's a [MASK] difference there [SEP] [MASK] is a significant difference there [MASK] it works in our favor. [SEP] [CLS] but i guess it [MAS

In [15]:
data_collator_word= lambda features: fn.whole_word_masking_data_collator(features, tokenizer)

In [16]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, lm_datasets['train'],lm_datasets['test'])

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [17]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 31.64


In [18]:
trainer.train()

  0%|          | 0/7760 [00:00<?, ?it/s]

{'loss': 2.6348, 'learning_rate': 9.621212121212123e-05, 'epoch': 1.0}


  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 2.249415636062622, 'eval_runtime': 3.0564, 'eval_samples_per_second': 898.114, 'eval_steps_per_second': 28.138, 'epoch': 1.0}
{'loss': 2.2792, 'learning_rate': 8.553719008264463e-05, 'epoch': 2.0}


  0%|          | 0/86 [00:00<?, ?it/s]

...

  0%|          | 0/86 [00:00<?, ?it/s]

{'eval_loss': 1.9845937490463257, 'eval_runtime': 3.3793, 'eval_samples_per_second': 812.304, 'eval_steps_per_second': 25.449, 'epoch': 10.0}


Overwriting existing adapter 'unipelt_source_mlm'.


{'train_runtime': 644.0416, 'train_samples_per_second': 385.224, 'train_steps_per_second': 12.049, 'train_loss': 2.1909817385919315, 'epoch': 10.0}


TrainOutput(global_step=7760, training_loss=2.1909817385919315, metrics={'train_runtime': 644.0416, 'train_samples_per_second': 385.224, 'train_steps_per_second': 12.049, 'train_loss': 2.1909817385919315, 'epoch': 10.0})

In [19]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/86 [00:00<?, ?it/s]

>>> Perplexity: 7.36


In [20]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [21]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

In [24]:
masked_sentence = "I bought the latest version of [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i bought the latest version of it.
>>> i bought the latest version of that.
>>> i bought the latest version of them.
>>> i bought the latest version of this.
>>> i bought the latest version of me.
